In [ ]:
import nbappinator as nbapp
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
# Globals that are used throughout the app

PAGES = ["Table", "Chart"]

DF = pd.DataFrame(np.random.uniform(1, 1000, size=(20, 10)), columns=[f"col_{r}" for r in range(0,10)])

def get_df() -> pd.DataFrame:
    return DF


In [ ]:
def update_click(component, action, args, app: nbapp.UiModel, caller: str):
    with app.messages: 
        try:
            print(f"update_click called, {action}")

            target_page = app.get_page(PAGES[0])
            # Sends stdout to the messages box underneath app. We'll usually use a logger here. 
            print(f"update_click called, {action}")

            # Updates the status box next to the button
            app.update_status(caller, "Updating DF", running=True)

            target_page.clear_page()  # Removes any previously displayed data from the page

            precision=app.get_values("c.precision")

            df = get_df()
            target_page.add_df(name="table1", df=df, precision = precision)

            app.update_status(caller, "Done creating DF", running=False)
            print("update_click done")
        except Exception as e:
            app.update_status(caller, f"Failure to update {e}", running=False)
            print(e)


def add_chart(component, action, args, app: nbapp.UiModel, caller: str):
    with app.messages: 
        # Get the current DF from the widgets
        app.update_status("b_chart", "Creating chart", running=False)
        target_page = app.get_page(PAGES[1])
        
        target_page.clear_page()  # Removes any previously displayed data from the page

        df = get_df()
        fig = px.line(df, y=df.columns)
        target_page.add_plotly_fig(name="chart1", fig=fig)
        app.update_status("b_chart", "Done creating chart", running=False)


In [ ]:
# Creates a one "tab" app, with a config section on top, and a dataframe grid below.
# Clicking "Update Precision" will redraw the dataframe grid with the selected precision

myapp = nbapp.TabbedUiModel(pages=PAGES, log_footer = "Messages", headers=["Config"])

config_page = myapp.get_page("Config")

config_page.add_select(name="c.precision", label="Precision", options=[0, 1, 2,3,4,5], value=2)
config_page.add_box(name="buttons_box", horiz=True)

config_page.add_button(override_page="buttons_box", name="b_update", label="Update Precision", action=update_click, status=True)
config_page.add_button(override_page="buttons_box", name="b_chart", label="Add Chart", action=add_chart, status=True)

myapp.get_page(PAGES[1]).add_textstatic(name="Notice", value="Click Add Chart to Populate This Page")

# Initialize
update_click(None, None, None, app=myapp, caller="b_update")

myapp.display()